In [1]:
!pip install --upgrade pip --quiet
!pip install "sagemaker>=2.48.0" --upgrade --quiet

In [8]:
from datetime import datetime
from pathlib import Path
from uuid import uuid4

import sagemaker
import boto3
import json
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
from botocore.exceptions import NoCredentialsError
from sagemaker.jumpstart.model import JumpStartModel

session = sagemaker.Session()
role = "AmazonSageMaker-ExecutionRole-20240618T160945"
runtime = boto3.client("sagemaker-runtime")
region = session.boto_region_name
endpoint_name='spectra-test'

display(f"Role: {role}")
display(f"Region: {region}")
display(sagemaker.__version__)


'Role: AmazonSageMaker-ExecutionRole-20240618T160945'

'Region: us-east-1'

'2.224.2'

In [15]:
# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'barghavani/MITRE_cybersecurity',
	'HF_TASK':'token-classification'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	transformers_version='4.37.0',
	pytorch_version='2.1.0',
	py_version='py310',
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1, # number of instances
	instance_type='ml.m5.xlarge',
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

------!

In [18]:
predictor.predict({
	"inputs": "what is vulnerability?",
})

[{'entity': 'LABEL_0',
  'score': 0.999825656414032,
  'index': 1,
  'word': 'what',
  'start': 0,
  'end': 4},
 {'entity': 'LABEL_0',
  'score': 0.9998394250869751,
  'index': 2,
  'word': 'Ġis',
  'start': 5,
  'end': 7},
 {'entity': 'LABEL_0',
  'score': 0.9998741149902344,
  'index': 3,
  'word': 'Ġvulnerability',
  'start': 8,
  'end': 21},
 {'entity': 'LABEL_0',
  'score': 0.9999207258224487,
  'index': 4,
  'word': '?',
  'start': 21,
  'end': 22}]

In [16]:
#model_id = "huggingface-llm-mistral-7b"
#model = JumpStartModel(model_id=model_id,     
#                       region=region,
#                        role=role)
#example_payloads = model.retrieve_all_examples()
#example_payloads

In [10]:
question = "what is owasp?"

input_data = {
	"inputs": {
		"source_sentence": "That is a happy person",
		"sentences": [
			"That is a happy dog",
			"That is a very happy person",
			"Today is a sunny day"
		]
	},
}
    

In [14]:
predictor.predict({
	"inputs": {
		"source_sentence": "That is a happy person",
		"sentences": [
			"That is a happy dog",
			"That is a very happy person",
			"Today is a sunny day"
		]
	},
})

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from primary and could not load the entire response body. See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/huggingface-pytorch-inference-2024-07-01-09-18-38-247 in account 452706865406 for more information.

In [11]:
body = json.dumps(input_data)



response = sagemaker_runtime.invoke_endpoint(
    EndpointName="huggingface-pytorch-inference-2024-07-01-09-18-38-247",
    ContentType='application/json',
    Accept='application/json',
    Body=body
)

result = json.loads(response['Body'].read().decode())

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from primary and could not load the entire response body. See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/huggingface-pytorch-inference-2024-07-01-09-18-38-247 in account 452706865406 for more information.

In [65]:
result[0]['generated_text']

'Hogamew and Lindbergha Ice'

In [22]:
try:
    #print(f"Deleting model: {model_name}")
    predictor.delete_model()
except Exception as e:
    print(f"{e}")


#print(f"Deleting endpoint: {endpoint_name}")
predictor.delete_endpoint()


One or more models cannot be deleted, please retry. 
Failed models: huggingface-pytorch-inference-2024-07-01-09-35-25-404
